# Semantic Guardrails

Guardrails are instructions sent to the LLM (often in the system prompt or through filters) that define what the agent should or shouldn’t say or do to keep its behavior within set boundaries.

## Semantic Guardrails vs. Prompt Guardrails

**Prompt guardrails** are rules written directly into an LLM’s system or developer prompts. They guide the model’s behavior by telling it what it should or shouldn’t say. For example, “Don’t discuss personal medical advice.” While effective, this approach has a few trade-offs:
- **Cost per request**: Each decision requires an API call
- **Latency**: Network round-trips add delay to every tool selection
- **Vulnerability to jailbreaks**: Cleverly phrased inputs can trick the model into ignoring or bypassing the rules

**Semantic guardrails** uses vector embeddings and similarity matching to block queries that match a blocked topic:
- **Speed**: Near-instantaneous tool selection through vector similarity
- **Cost-effective**: No API costs after initial setup
- **Resistant to jailbreaks**: Because semantic guardrails work outside the LLM and detect meaning directly, they’re much harder to bypass with prompt tricks or rewording.

## How It Works

Creating the references and storing them in the vector database (Redis):
1. Reference examples of text are generated for each topic we want to semantically block.
2. Using an embedding model, we convert these references into embeddings (vector representation)
3. These references are stored in Redis.

Checking topic:
1. Using the same embedding model, we convert the user prompt into an embedding (vector representation)
2. We use this embedding to perform semantic search in the vector database to retrieve the most similar reference to the topics we're trying to block
3. If the most similar reference is similar enough, we assume that the referring topic should be blocked.

## Using RedisVL (Vector Library)

RedisVL is a library that makes working with vector search easy with Redis by providing abstractions to common vector search use cases out of the box. In this notebook, we will use the *Semantic Routing* abstraction whose purpose is to classify text in the same fashion described in the previous section.

## Resources
- [RedisVL Java GitHub Repository](https://github.com/redis/redis-vl-java)
- [RedisVL Java Documentation](https://redis.github.io/redis-vl-java/redisvl/current/index.html)
- [RedisVL Python GitHub Repository](https://github.com/redis/redis-vl-python)
- [RedisVL Python Documentation](https://docs.redisvl.com/en/latest/)
- [Redis AI Resources Repository](https://github.com/redis-developer/redis-ai-resources)
- [Redis Query Engine Documentation](https://redis.io/docs/latest/develop/ai/search-and-query/)

## Running Redis

There are several options one can follow to have a running instance of Redis. For the sake of simplicity, in this notebook, we will run it in a Docker container.

For production where high-availability and reliability is a concern, we recommend using [Redis Cloud](https://cloud.redis.io/).

A free database can be spun up in Redis Cloud.

### Running Redis in a Docker Container using TestContainers

**Docker containers** are lightweight, portable environments that package an application and all its dependencies so it runs consistently across different systems. **Testcontainers** is a library that lets us run lightweight, disposable Docker containers for integration testing, so you can test against real services like databases or message queues without complex setup.

Make sure you have Docker installed: [install Docker](https://www.docker.com/get-started/).

#### Installing dependencies

In [1]:
@file:DependsOn("org.testcontainers:testcontainers:2.0.2")

#### Configuring a generic Redis Container

In [2]:
import org.testcontainers.containers.GenericContainer
import org.testcontainers.utility.DockerImageName

class RedisContainer : GenericContainer<RedisContainer>(DockerImageName.parse("redis:latest")) {
    init {
        withExposedPorts(6379)
    }
}

#### Creating a Docker network

This is necessary because later on this notebook we will spin up a Redis Insight container that needs to be in the same network.

In [3]:
import org.testcontainers.containers.Network

val network = Network.newNetwork()
val networkAlias = "redis-network"

#### Start a Redis Container

In [4]:
val networkAlias = "redis"
val redis = RedisContainer().withNetwork(network).withNetworkAliases(networkAlias)
redis.start()

val host = redis.host
val port = redis.getMappedPort(6379)
println("Redis 8 started at $host:$port")

Redis 8 started at localhost:54468


## Implementing our Semantic Guardrail

### Installing dependencies

As mentioned in the beginning, we will use RedisVL's semantic routing abstraction to implement our semantic guardrail. Therefore, we will need to add RedisVL as a dependency.

In [5]:
@file:DependsOn("com.redis:redisvl:0.0.1")

### Setting up a vectorizer

In RedisVL, embedding models are called vectorizers. This is because embeddings are vector representations. The vectorizer is responsible for converting text into numerical vector representations that capture semantic meaning.

This vectorizer will be passed on to our semantic routing that will convert the references and the text we're trying to block into vectors under the hood.

RedisVL provides several vectorizer options such as OpenAI and VertexAI, but for this example, we will be HuggingFace's `all-MiniLM-L6-v2` vectorizer because it's open source, lightweight, and free to use.

In [6]:
import com.redis.vl.utils.vectorize.SentenceTransformersVectorizer

val vectorizer = SentenceTransformersVectorizer("Xenova/all-MiniLM-L6-v2")


// Testing our vectorizer
// all-MiniLM-L6-v2 is an embedding model that produces vectors of 384 dimensions, therefore we will 384 numbers printed on the screen.
// Embedding models are deterministic. It doesn't matter how many times we run this cell, the same numbers will always be produced for the same string.

val embedding = vectorizer.embed("What is the capital city of the Netherlands?")

println(embedding.joinToString())

0.10366548, 0.06542453, -0.04904806, 0.035133816, -0.030148711, -0.048898157, -0.02108736, 0.0019588028, -0.05460191, 0.027000071, 0.0186685, -0.12342901, -0.07914663, -0.0302804, -0.056598365, -0.039736673, 0.030802587, 0.005838588, 0.085851155, -0.032130066, -0.0071115145, -0.033734083, 0.100847885, -0.06491691, 0.014052424, 0.036977015, 0.04544064, -0.014863417, 0.011651148, -0.04714538, 0.019530838, -0.06317588, 0.027103335, -0.032490354, -0.06364442, 0.0034463818, -0.022536488, 0.046401046, 0.029528277, 0.023609689, 0.026152493, -0.025078116, -0.01031126, -0.0460871, -0.030701958, -0.011587745, -0.046117976, 0.0654084, -0.0105588185, -0.030012755, 0.08957275, -0.06994565, -0.07410133, -0.030177299, -0.0072215544, 0.03257758, -0.08564555, 0.06931229, 0.011757878, -0.017046366, 0.006678676, 0.005762717, -0.09732431, 0.04363133, 0.09194445, 0.0023713051, 0.032854725, 0.043560334, -0.09262396, -0.0036028812, -0.00783084, -0.051787496, 0.020866683, -0.08783279, 0.008077556, -0.06189656

## Defining the Block Route

The blocking route represents all topics we may want our agent to not respond to. We define the route by providing example phrases that represent different ways users may refer to the topics that should be blocked.

Each route contains:

- **Route name**: An identifier for this classification category
- **Reference examples**: Sample text that represents the category you want to classify
- **Distance threshold**: How similar new text must be to the references to match the route

In [7]:
import com.redis.vl.extensions.router.Route

val blockRoute = Route.builder()
    .name("block_route")
    .references(
        listOf(
            "things about aliens",
            "corporate questions about agile",
            "anything about the S&P 500"
        )
    )
    .build()

### Creating the router

The SemanticRouter is the central component that orchestrates the blocking process. It combines your routes, vectorizer, and Redis connection to provide fast semantic classification capabilities.

In [8]:
import com.redis.vl.extensions.router.SemanticRouter
import redis.clients.jedis.HostAndPort
import redis.clients.jedis.UnifiedJedis

// Configure the connection to Redis
val jedis = UnifiedJedis(HostAndPort(host, port))

val router = SemanticRouter.builder()
    .name("block-router")
    .jedis(jedis)
    .vectorizer(vectorizer)
    .routes(listOf(blockRoute)
    ).build()

## Testing our semantic guardrail solution

In [9]:
val userQuery = "Is scrum a good framework?"

val routeMatch = router.route(userQuery)

// This query should be blocked
println(routeMatch)

RouteMatch(name=block_route, distance=0.486094415188)


In [10]:
val userQuery = "Tell me a joke with an alien"

val routeMatch = router.route(userQuery)

// This query should be blocked
println(routeMatch)

RouteMatch(name=block_route, distance=0.417759597301)


In [11]:
val userQuery = "Help me solve a problem"

val routeMatch = router.route(userQuery)

// This query should be allowed
println(routeMatch)

RouteMatch(name=null, distance=null)


## Redis Insight

Redis Insight is a visual tool that helps you explore, monitor, and optimize your Redis data and performance through an easy-to-use interface.

It can be downloaded and run locally in your machine or be run in a Docker container. To make this recipe self-contained and straightforward, we're going to run it in a Docker container using Test Containers.

### Configuring a generic Redis Insight Container

In [12]:
import org.testcontainers.containers.GenericContainer
import org.testcontainers.containers.wait.strategy.Wait
import org.testcontainers.utility.DockerImageName

class RedisInsightContainer : GenericContainer<RedisInsightContainer>(
    DockerImageName.parse("redis/redisinsight:latest") // or latest stable version
) {
    init {
        withExposedPorts(5540)
        withEnv("RI_REDIS_HOST", "redis")
        withEnv("RI_REDIS_PORT", "6379") // Since this will run in the same Docker network, we don't need to set the mapped port for the Redis Server
        withEnv("RI_REDIS_ALIAS", "Local Redis")
        withEnv("RI_REDIS_USERNAME", "default")
        withEnv("RI_REDIS_PASSWORD", "")
        withEnv("RI_REDIS_TLS", "FALSE")

        waitingFor(Wait.forHttp("/").forPort(5540))
    }

    fun getUiUrl(): String = "http://${host}:${getMappedPort(5540)}"
}

### Starting the Redis Insight container

In [13]:
val redisInsight = RedisInsightContainer().withNetwork(network)
redisInsight.start()

println("RedisInsight UI: ${redisInsight.getUiUrl()}")

RedisInsight UI: http://localhost:54475


When accessing Redis Insight for the first time, you will have to agree with the user agreement:

<img src="./readme-assets/1_1_redis_insight_user_agg.png" alt="" width="500">

After agreeing, the list of configured databases will show up. In this case, there'll be only one: `Local Redis`.

<img src="./readme-assets/1_2_redis_insight_list_of_db.png" alt="" width="500">

By clicking on `Tree View` we can organize the keys by keyspace. This will make it easier to visualize all keys in Redis Insight:

<img src="./readme-assets/1_3_redis_insight_tree_view.png" alt="" width="500">

The `block-router:route_config` key holds the configuration of the router (guardrail in our case) - We can see its name, vectorizer, routes and some configuration:

<img src="./readme-assets/3_4_redis_insight_route_config.png" alt="" width="1000">

In the `block-router:block_route:` keyspace, we can see the details of each vectorized reference, including their respective vector representations:

<img src="./readme-assets/3_5_redis_insight_reference_details.png" alt="" width="1000">

Make sure you change from `Unicode` to `Vector 32-bit` to see the vectors as numbers instead of a bytearray:

<img src="./readme-assets/1_6_redis_insight_vector_32bit.png" alt="" width="500">

This will be a long list of 384 floating points.

On Redis Insight Workbench we can send commands directly to our Redis instance:

<img src="./readme-assets/1_7_redis_insight_workbench.png" alt="" width="300">

If we send the command `FT.INFO 'block-router'` we can see the index that was created by RedisVL to be able to perform semantic search efficiently using the [Redis Query Engine](https://redis.io/docs/latest/develop/ai/search-and-query/)

<img src="./readme-assets/3_8_redis_insight_index.png" alt="" width="1000">

## Spinning down Docker containers

Finally, once we're done, let's clean up all the resources we created for our recipe:

In [14]:
redis.stop()
redisInsight.stop()
network.close()